In [29]:
import pandas as pd
import numpy as np
from collections import Counter
from numpy import where
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from random import randrange, uniform

In [3]:
credit_train = pd.read_csv('./dataset/creditcard.csv')
is_fraud = (credit_train['Class'] == 1)
non_fraud = credit_train[~is_fraud]
fraud = credit_train[is_fraud]

In [4]:
X_maj = non_fraud.drop(columns=['Class']).copy()
y_maj = non_fraud['Class'].copy()
X_min = fraud.drop(columns=['Class']).copy()
y_min = non_fraud['Class'].copy()

In [5]:
x1_id = y_maj.sample().index.item()
x1 = X_maj.iloc[x1_id]
print(x1_id)

108353


In [6]:
knn = KNeighborsClassifier(n_neighbors=5)

In [7]:
import time
tic = time.perf_counter()
knn.fit(X_maj.to_numpy(), y_maj.to_numpy())
toc = time.perf_counter()
print(f"{toc - tic:0.4f} seconds")

0.0197 seconds


In [8]:
distances, indices = knn.kneighbors(x1.to_numpy().reshape(1, -1))
print(indices)

[[108353 108362 108322 108315 108344]]


In [9]:
knn = NearestNeighbors(n_neighbors=6)

In [10]:
import time
tic = time.perf_counter()
knn.fit(X_maj.to_numpy(), y_maj.to_numpy())
toc = time.perf_counter()
print(f"{toc - tic:0.4f} seconds")

0.0092 seconds


In [11]:
distances, indices = knn.kneighbors(x1.to_numpy().reshape(1, -1))
print(indices)

[[108353 108362 108322 108315 108344 108310]]


In [44]:
ratio = 0.9
n = len(X_maj) - len(X_min)
n_smote = round(n * ratio)
n_smute = n - n_smote
print(n_smute)

28382


In [79]:
# X'_maj <- X_maj
X_maj_prime = X_maj.copy()
y_maj_prime = y_maj.copy()
x3_y = y_maj_prime.sample().copy()
print(len(X_maj_prime))

tic = time.perf_counter()
while((len(X_maj) - len(X_maj_prime)) < n_smute):
    x1_d = X_maj_prime.sample().index.item()
    x1 = X_maj_prime.iloc[x1_id]
    knn.fit(X_maj_prime.to_numpy(), y_maj_prime.to_numpy())
    distances, indices = knn.kneighbors(x1.to_numpy().reshape(1, -1))
    indices = indices[0][1:6]
    x2_id = indices[randrange(5)]
    x2 = X_maj_prime.iloc[x2_id]
    r = uniform(0, 1)
    x3 = x1 + (r * x2)

    X_maj_prime = X_maj_prime.drop([x1_id, x2_id])
    y_maj_prime = y_maj_prime.drop([x1_id, x2_id])

    X_maj_prime = X_maj_prime.append(x3, ignore_index=True)
    y_maj_prime = y_maj_prime.append(x3_y, ignore_index=True)

toc = time.perf_counter()
print(f"{toc - tic:0.4f} seconds")


284315
2603.1568 seconds


In [81]:
print(len(X_maj))
print(len(X_maj_prime))

284315
255933


In [ ]:
def SMUTE(X_maj, y_maj, n_smute):
    X_maj_prime = X_maj.copy()
    y_maj_prime = y_maj.copy()
    x3_y = y_maj_prime.sample().copy()
    
    knn = NearestNeighbors(n_neighbors=6)
    
    tic = time.perf_counter()
    while((len(X_maj) - len(X_maj_prime)) < n_smute):
        x1_id = X_maj_prime.sample().index.item()
        x1 = X_maj_prime.iloc[x1_id]
        knn.fit(X_maj_prime.to_numpy(), y_maj_prime.to_numpy())
        distances, indices = knn.kneighbors(x1.to_numpy().reshape(1, -1))
        indices = indices[0][1:6]
        x2_id = indices[randrange(5)]
        x2 = X_maj_prime.iloc[x2_id]
        r = uniform(0, 1)
        x3 = x1 + (r * x2)

        X_maj_prime = X_maj_prime.drop([x1_id, x2_id])
        y_maj_prime = y_maj_prime.drop([x1_id, x2_id])

        X_maj_prime = X_maj_prime.append(x3, ignore_index=True)
        y_maj_prime = y_maj_prime.append(x3_y, ignore_index=True)

    toc = time.perf_counter()
    print(f"{toc - tic:0.4f} seconds")
    
    return X_maj_prime, y_maj_prime

In [ ]:
def SMOTE(X_min, y_min, n_smote):
    X_min_prime = X_min.copy()
    y_min_prime = y_min.copy()
    x3_y = y_min_prime.sample().copy()
    
    knn = NearestNeighbors(n_neighbors=6)
    
    tic = time.perf_counter()
    while((len(X_min) - len(X_min_prime)) < n_smote):
        x1_id = X_min_prime.sample().index.item()
        x1 = X_min_prime.iloc[x1_id]
        knn.fit(X_min_prime.to_numpy(), y_min_prime.to_numpy())
        _, indices = knn.kneighbors(x1.to_numpy().reshape(1, -1))
        indices = indices[0][1:6]
        x2_id = indiced[randrange(5)]
        x2 = X_maj_prime.iloc[x2_id]
        r = uniform(0, 1)
        x3 = x1 + (r * x2)
        
        X_min_prime = X_min_prime.append(x3, ignore_index=True)
        y_min_prime = y_min_prime.append(x3_y, ignore_index=True)
    
    toc = time.perf_counter()
    print(f"{toc - tic:0.4f} seconds")
    
    return X_min_prime, y_min_prime

In [ ]:
def CSMOUTE(majority, minority, ratio):
    X_maj = majority.drop(columns=['Class']).copy()
    y_maj = majority['Class'].copy()
    X_min = minority.drop(columns=['Class']).copy()
    y_min = minority['Class'].copy()
    
    n = len(X_maj) - len(X_min)
    n_smote = round(n * ratio)
    n_smute = n - n_smote
    
    X_min_prime, y_min_prime = SMOTE(X_min, y_min, n_smote)
    X_maj_prime, y_maj_prime = SMUTE(X_maj, y_maj, n_smute)
    
    return X_maj_prime, y_maj_prime, X_min_prime, y_min_prime